
### Entrada
- x1: Local da Coleta em km
- x2: Nivel da Mare em m
- x3: Umidade do Ar (%)
- x4: Índice Pluvimétrico (mm)¹
- x5: Temperatura (°C)

### Saida
- y1: pH
- y2: Condutividade Elétrica (mS.cmˉ¹)
- y3: Oxigênio Dissolvido (mg.Lˉ¹)
- y4: Sólidos Totais Dissolvidos (ppm)


In [1]:
!pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [2]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

Cloning into 'tf-levenberg-marquardt'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 53 (delta 31), reused 29 (delta 14), pack-reused 0
Receiving objects: 100% (53/53), 32.99 KiB | 16.50 MiB/s, done.
Resolving deltas: 100% (31/31), done.


In [3]:
import numpy as np
import pandas as pd

def create_dataframe(output, label):
    df = pd.read_excel("../../../data.xlsx")
    x1, x2, x3, x4, x5  = df['x1'], df['x2'], df['x3'], df['x4'], df['x5']

    input = np.vstack([x1, x2, x3, x4, x5 ]).T
    output = np.array(df[output])
    return output, input

output, input = create_dataframe(output="y1", label="ph")

/tmp/ipykernel_233876/4275043310.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer
        
        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

2024-03-24 18:15:46.563813: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-24 18:15:46.634102: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 18:15:46.634143: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 18:15:46.635534: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-24 18:15:46.643451: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-24 18:15:46.644498: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error


class TrainWithSmallDataset:
    def __init__(self, batch_size=1000):
        self.batch_size = batch_size
        self.betters = []
    
    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output):
      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)
 
 
    
    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)

          metrics = {
                          'r2': r2,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics

In [6]:
from itertools import product
import pickle

class Tester:
  def __init__(self, input, output, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input, self.output = input, output
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=5,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)

  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path) 

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)


  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] >= boundarie): # should be <= to descending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

# Init
  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input, self.output)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults(sort_by, header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)

In [7]:
tester = Tester(input, output, run_times=25, dataset_run_times=10)

In [8]:
tester.Loop(sort_by='r2',
            boundarie=0.8,
            hidden_sizes = [[15], [30], [35], [40], [55], [60]],
            regularizers=[0.02, 0.1],
            learning_rate=[0.02, 0.1])

Testando combinacao1: Hidden Size=[15], regularizer=0.02, learning_rate=0.02
+++++++++++ [1_0] | 1 ++++++++++++++++++
Elapsed time:  1.6660514899995178
Stopped at epoch:  27
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++
Elapsed time:  0.8472969879994707
Stopped at epoch:  19
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Elapsed time:  0.23469809200014424
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Elapsed time:  0.4194862389995251
Stopped at epoch:  8
1/1 [==============================] - 0s 20ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Elapsed time:  0.24111886500031687
Stopped at epoch:  6
1/1 [==============================] - 0s 21ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Elapsed time:  0.3138441760002024
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_0] | 7 +++++++++

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_5_9_0,0.817837,0.468997,0.125579,0.358062,0.034079,0.050576,0.081954,0.065342
model_5_9_1,0.819314,0.479123,0.133800,0.366639,0.033802,0.049612,0.081183,0.064469
model_5_9_2,0.825475,0.525932,0.163522,0.402707,0.032650,0.045154,0.078398,0.060798
model_5_9_14,0.834025,0.693402,0.081333,0.450056,0.031050,0.029203,0.086101,0.055978
model_5_9_3,0.836374,0.621161,0.204472,0.467628,0.030611,0.036083,0.074560,0.054190
model_5_9_13,0.836957,0.703642,0.110861,0.467922,0.030502,0.028227,0.083333,0.054160
model_5_9_12,0.838851,0.710219,0.130182,0.479555,0.030147,0.027601,0.081522,0.052976
model_5_9_4,0.840004,0.650721,0.219916,0.488960,0.029932,0.033268,0.073112,0.052018
model_5_9_11,0.841773,0.717242,0.163270,0.497371,0.029601,0.026932,0.078421,0.051162
model_5_9_5,0.842184,0.680388,0.214709,0.501401,0.029524,0.030442,0.073600,0.050752


DataFrame salvo em ../results/metrics_5_9
Testando combinacao6: Hidden Size=[30], regularizer=0.02, learning_rate=0.1
+++++++++++ [6_0] | 1 ++++++++++++++++++
Elapsed time:  1.1739331270000548
Stopped at epoch:  21
1/1 [==============================] - 0s 21ms/step
+++++++++++ [6_0] | 2 ++++++++++++++++++
Elapsed time:  0.30762606200005393
Stopped at epoch:  7
1/1 [==============================] - 0s 20ms/step
+++++++++++ [6_0] | 3 ++++++++++++++++++
Elapsed time:  0.24547580699982063
Stopped at epoch:  6
1/1 [==============================] - 0s 18ms/step
+++++++++++ [6_0] | 4 ++++++++++++++++++
Elapsed time:  0.29071428799943533
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [6_0] | 5 ++++++++++++++++++
Elapsed time:  0.3026444970000739
Stopped at epoch:  6
1/1 [==============================] - 0s 17ms/step
+++++++++++ [6_0] | 6 ++++++++++++++++++
Elapsed time:  0.3253804879996096
Stopped at epoch:  6
1/1 [==============================] - 0s 

2024-03-24 18:29:19.912189: W tensorflow/core/data/root_dataset.cc:342] Optimization loop failed: CANCELLED: Operation was cancelled


Elapsed time:  0.5986265020001156
Stopped at epoch:  6
1/1 [==============================] - 0s 21ms/step
+++++++++++ [7_8] | 21 ++++++++++++++++++
Elapsed time:  0.39147874399986904
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [7_8] | 22 ++++++++++++++++++
Elapsed time:  0.40106003700020665
Stopped at epoch:  6
1/1 [==============================] - 0s 24ms/step
+++++++++++ [7_8] | 23 ++++++++++++++++++
Elapsed time:  0.36897672500072076
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [7_8] | 24 ++++++++++++++++++
Elapsed time:  0.35384588000033546
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [7_8] | 25 ++++++++++++++++++
Elapsed time:  0.325222174999908
Stopped at epoch:  6
1/1 [==============================] - 0s 20ms/step
+++++++++++ [7_9] | 1 ++++++++++++++++++
Elapsed time:  0.23570436499994685
Stopped at epoch:  6
1/1 [==============================] - 0s 24ms/s

...

DataFrame salvo em ../results/metrics_18_9
Testando combinacao19: Hidden Size=[55], regularizer=0.1, learning_rate=0.02
+++++++++++ [19_0] | 1 ++++++++++++++++++
Elapsed time:  0.8530259740000474
Stopped at epoch:  15
1/1 [==============================] - 0s 22ms/step
+++++++++++ [19_0] | 2 ++++++++++++++++++
Elapsed time:  0.27217292100067425
Stopped at epoch:  6
1/1 [==============================] - 0s 21ms/step
+++++++++++ [19_0] | 3 ++++++++++++++++++
Elapsed time:  0.26140128999941226
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [19_0] | 4 ++++++++++++++++++
Elapsed time:  0.9182635830002255
Stopped at epoch:  24
1/1 [==============================] - 0s 16ms/step
+++++++++++ [19_0] | 5 ++++++++++++++++++
Elapsed time:  0.26612603700050386
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [19_0] | 6 ++++++++++++++++++
Elapsed time:  0.22012127100060752
Stopped at epoch:  6
1/1 [===========================

2024-03-24 19:01:47.236608: W tensorflow/core/data/root_dataset.cc:342] Optimization loop failed: CANCELLED: Operation was cancelled


1/1 [==============================] - 0s 26ms/step
+++++++++++ [19_5] | 5 ++++++++++++++++++
Elapsed time:  0.35552559700045094
Stopped at epoch:  6
1/1 [==============================] - 0s 17ms/step
+++++++++++ [19_5] | 6 ++++++++++++++++++
Elapsed time:  0.279464425999322
Stopped at epoch:  6
1/1 [==============================] - 0s 19ms/step
+++++++++++ [19_5] | 7 ++++++++++++++++++
Elapsed time:  0.26038631800111034
Stopped at epoch:  6
1/1 [==============================] - 0s 17ms/step
+++++++++++ [19_5] | 8 ++++++++++++++++++
Elapsed time:  0.28232439199928194
Stopped at epoch:  6
1/1 [==============================] - 0s 21ms/step
+++++++++++ [19_5] | 9 ++++++++++++++++++
Elapsed time:  0.1744245090012555
Stopped at epoch:  6
1/1 [==============================] - 0s 24ms/step
+++++++++++ [19_5] | 10 ++++++++++++++++++
Elapsed time:  0.21880359999886423
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [19_5] | 11 ++++++++++++++++++
Elapsed

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_23_9_0,0.803067,0.946709,0.168672,0.720993,0.036842,0.011712,0.084046,0.045751
model_23_9_1,0.805845,0.935670,0.168758,0.713186,0.036322,0.014137,0.084037,0.047031
model_23_9_2,0.808330,0.923973,0.169141,0.704998,0.035857,0.016708,0.083998,0.048374
model_23_9_3,0.810553,0.911888,0.169483,0.696524,0.035441,0.019364,0.083964,0.049764
model_23_9_4,0.812536,0.899452,0.169854,0.687806,0.035070,0.022097,0.083926,0.051193
model_23_9_5,0.814302,0.886654,0.170560,0.678932,0.034740,0.024909,0.083855,0.052649
model_23_9_6,0.815879,0.873685,0.171317,0.669949,0.034445,0.027760,0.083779,0.054122
model_23_9_7,0.817271,0.860466,0.172328,0.660863,0.034184,0.030665,0.083676,0.055611
model_23_9_8,0.818511,0.847103,0.173726,0.651787,0.033952,0.033601,0.083535,0.057100
model_23_9_9,0.819616,0.833633,0.175396,0.642715,0.033746,0.036562,0.083366,0.058587


DataFrame salvo em ../results/metrics_23_9
Testando combinacao24: Hidden Size=[60], regularizer=0.1, learning_rate=0.1
+++++++++++ [24_0] | 1 ++++++++++++++++++
Elapsed time:  1.2527065359990956
Stopped at epoch:  12
1/1 [==============================] - 0s 17ms/step
+++++++++++ [24_0] | 2 ++++++++++++++++++
Elapsed time:  0.4510158590001083
Stopped at epoch:  7
1/1 [==============================] - 0s 17ms/step
+++++++++++ [24_0] | 3 ++++++++++++++++++
Elapsed time:  0.359812681999756
Stopped at epoch:  6
1/1 [==============================] - 0s 21ms/step
+++++++++++ [24_0] | 4 ++++++++++++++++++
Elapsed time:  0.4682865890008543
Stopped at epoch:  7
1/1 [==============================] - 0s 22ms/step
+++++++++++ [24_0] | 5 ++++++++++++++++++
Elapsed time:  0.21476881099988532
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [24_0] | 6 ++++++++++++++++++
Elapsed time:  0.5694786089989066
Stopped at epoch:  8
1/1 [==============================] -